# 해야할 것

`1` BERTopic model을 불러와서 학습

`2` 모델 학습 후 각 기사 데이터의 토픽 추론(`BertTopic`)

`3` 추론한 토픽 컬럼을 생성

`4` 사용자가 언어를 선택하면 그 언어에 맞게끔 데이터가 바뀌는 함수 작성

`5` 기사의 핵심어 산출을 위한 `KeyBERT` model load

`6` 사용자가 해당 기사의 제목을 선택하면 요약문(`KoBART`)과 원문을 함께 보이게 하기 위해서 `데이터 프레임`을 `return` 하는 함수 작성

`7` 그 후 사용자가 토픽, 날짜, 지역을 선택하면 해당 지역의 기사를 위험도(`일단 내가 임의로 설정`)순으로 기사가 보이게끔

`8` 그리고 사용자가 어떤 지역의 언어를 선택하면 그 언어로 핵심어, 요약문, 원문을 번역해주는 함수 작성(`mBart-50`)

## 1. 데이터 로드

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/Hackathon/Newsdata(합본)_20231001-20231101.xlsx")

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9911 entries, 0 to 9910
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  2 non-null      object
 1   일자          9911 non-null   int64 
 2   제목          9911 non-null   object
 3   뉴스 본문       9911 non-null   object
 4   URL         9823 non-null   object
dtypes: int64(1), object(4)
memory usage: 387.3+ KB


`-` 쓸데없는 컬럼 삭제

In [ ]:
df.drop('Unnamed: 0', axis = 1, inplace =True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9911 entries, 0 to 9910
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   일자      9911 non-null   int64 
 1   제목      9911 non-null   object
 2   뉴스 본문   9911 non-null   object
 3   URL     9823 non-null   object
dtypes: int64(1), object(3)
memory usage: 309.8+ KB


`-` 일자를 `날짜 데이터 ` 형식으로 변경

In [ ]:
df.일자 = [str(i)[0:4] + "-" + str(i)[4:6] + "-" + str(i)[6:] for i in df.일자]
df.일자 = pd.to_datetime(df.일자)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9911 entries, 0 to 9910
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   일자      9911 non-null   datetime64[ns]
 1   제목      9911 non-null   object        
 2   뉴스 본문   9911 non-null   object        
 3   URL     9823 non-null   object        
dtypes: datetime64[ns](1), object(3)
memory usage: 309.8+ KB


In [ ]:
df.head()

,일자,제목,뉴스 본문,URL
0,2023-11-01,"군복 입은 아들의 사진, 이 대통령을 강직하게 만들었다[정미경의 이런영어 저런미국]",“아들에게 자랑스러운 아버지가 되겠다” 백악관 집무실에 놓인 사진에 담긴 사연 대통...,https://www.donga.com/news/Inter/article/all/2...
1,2023-11-01,"""광주학생들의 저항정신 곳곳에 살아 숨 쉰다""",광주서부교육지원청과 무등일보가 공동 주최한 94주년 광주학생독립운동 발상지 현장취재...,http://www.mdilbo.com/detail/csATmn/705819
2,2023-11-01,[소감문] “조국 위해 분연히 일어났던 선조들의 뜻 가슴에 새길래요”,무등일보는 지난 27일 광주서부교육지원청과 공동으로 제94주년 학생의 날을 맞아 광...,http://www.mdilbo.com/detail/csATmn/705817
3,2023-11-01,"이승만 양자 이인수 박사 별세 ""父 뜻처럼 北동포 자유 누리길""",이승만 전 대통령 양자인 이인수 박사가 지난 9월 1일 오전 서울 강북구 수유동 4...,https://www.joongang.co.kr/article/25204088
4,2023-11-01,JMS 정명석 ‘법관 기피 신청’ 최종 기각 재판 재개 전망,기독교복음선교회(일명 JMS) 총재인 정명석(왼쪽)과 JMS 2인자로 알려진 김지선...,https://www.khan.co.kr/national/national-gener...


## 2. 전처리 (생략)

`-` 아직 형태소 분석기 뭐 쓸지 안정했으니 생략하자.

`-` [전처리+모델링 참고링크](https://gangcheol.github.io/mysite2/posts/DX/2023-10-26-02.%20BERTopic.html#mecab-%EC%84%A4%EC%B9%98)

## 3.  모델링

***

# KeyBERT 핵심어 추출(`다시 로컬로 넘어옴`)

In [ ]:
import pandas as pd

In [ ]:
# df = pd.read_excel("rowdata(size=100).xlsx")

FileNotFoundError: ignored

In [ ]:
df.head()

,일자,제목,뉴스 본문,URL
0,2023-11-01,"군복 입은 아들의 사진, 이 대통령을 강직하게 만들었다[정미경의 이런영어 저런미국]",“아들에게 자랑스러운 아버지가 되겠다” 백악관 집무실에 놓인 사진에 담긴 사연 대통...,https://www.donga.com/news/Inter/article/all/2...
1,2023-11-01,"""광주학생들의 저항정신 곳곳에 살아 숨 쉰다""",광주서부교육지원청과 무등일보가 공동 주최한 94주년 광주학생독립운동 발상지 현장취재...,http://www.mdilbo.com/detail/csATmn/705819
2,2023-11-01,[소감문] “조국 위해 분연히 일어났던 선조들의 뜻 가슴에 새길래요”,무등일보는 지난 27일 광주서부교육지원청과 공동으로 제94주년 학생의 날을 맞아 광...,http://www.mdilbo.com/detail/csATmn/705817
3,2023-11-01,"이승만 양자 이인수 박사 별세 ""父 뜻처럼 北동포 자유 누리길""",이승만 전 대통령 양자인 이인수 박사가 지난 9월 1일 오전 서울 강북구 수유동 4...,https://www.joongang.co.kr/article/25204088
4,2023-11-01,JMS 정명석 ‘법관 기피 신청’ 최종 기각 재판 재개 전망,기독교복음선교회(일명 JMS) 총재인 정명석(왼쪽)과 JMS 2인자로 알려진 김지선...,https://www.khan.co.kr/national/national-gener...


In [ ]:
!pip install keybert

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 76.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 89.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 87.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.6 MB/s eta 0:00:00
  Created wheel for keybert: filename=keybert-0.8.3-py3-none-any.whl size=39126 sha256=ad56ca09b7a9309df01509938b0558f6109becfe283085ccc741bcc924c56e95
  Stored in directory: /root/.cache/pip/wheels/70/88/07/1a3bc11fd1dd5f89924a02dcbca89a3015e25e8faa31f904dc
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl 

In [ ]:
from keybert import KeyBERT
from transformers import BartForConditionalGeneration

`-` pretrained 된 `digit82/kobart-summarization`을 기본 모델로 만들고 Keybert 객체로 만들어준다.

In [ ]:
model = BartForConditionalGeneration.from_pretrained('digit82/kobart-summarization')
kw_model = KeyBERT(model)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


`-` 일단 1개 데이터

In [ ]:
text = df["뉴스 본문"][0]

In [ ]:
keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=15)
keywords

[('대통령들의', 0.622),
 ('대통령', 0.5893),
 ('백악관', 0.5888),
 ('대통령처럼', 0.5828),
 ('대통령에게', 0.5701),
 ('미국인의', 0.5605),
 ('트럼프', 0.5503),
 ('대통령도', 0.5322),
 ('대통령입니다', 0.5313),
 ('오바마', 0.5105),
 ('백악관에는', 0.4975),
 ('대통령은', 0.4879),
 ('미국적인', 0.4796),
 ('연설', 0.4698),
 ('미국을', 0.4584)]

`-` 확인했으니 루프로 모든 데이터에 적용한다음에 컬럼 만들어서 붙이자.

In [ ]:
keyword_list = []

In [ ]:
for i in range(len(df)) :
    text = df["뉴스 본문"][i]
    temp = [i[0] for i in keywords]
    keyword_list.append(temp)

## 사용자 정의함수 작성

In [ ]:
def keyword_extract(df) :
    model = BartForConditionalGeneration.from_pretrained('digit82/kobart-summarization')
    kw_model = KeyBERT(model)
    keyword_list = []
    for i in range(len(df)) :
        text = df["뉴스 본문"][i]
        keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=15)
        temp = [i[0] for i in keywords]
        keyword_list.append(temp)
    df["keyword"] = keyword_list
    df["keyword"] = [" ".join(i) for i in df["keyword"]]
    df["doc_id"] = ["article" +str(i) for i in range(len(total_data))]
    return df

In [ ]:
# df = keyword_extract(df)
df = df.head(3)

NameError: ignored

In [ ]:
df = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/Hackathon/keyword포함.xlsx")

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9911 entries, 0 to 9910
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   일자       9911 non-null   datetime64[ns]
 1   제목       9911 non-null   object        
 2   뉴스 본문    9911 non-null   object        
 3   URL      9823 non-null   object        
 4   keyword  9911 non-null   object        
dtypes: datetime64[ns](1), object(4)
memory usage: 387.3+ KB


In [ ]:
df["doc_id"] = ["article" +str(i) for i in range(len(df))]

In [ ]:
df.head()

,일자,제목,뉴스 본문,URL,keyword,doc_id
0,2023-11-01,"군복 입은 아들의 사진, 이 대통령을 강직하게 만들었다[정미경의 이런영어 저런미국]",“아들에게 자랑스러운 아버지가 되겠다” 백악관 집무실에 놓인 사진에 담긴 사연 대통...,https://www.donga.com/news/Inter/article/all/2...,대통령들의 대통령 백악관 대통령처럼 대통령에게 미국인의 트럼프 대통령도 대통령입니다...,article0
1,2023-11-01,"""광주학생들의 저항정신 곳곳에 살아 숨 쉰다""",광주서부교육지원청과 무등일보가 공동 주최한 94주년 광주학생독립운동 발상지 현장취재...,http://www.mdilbo.com/detail/csATmn/705819,광주학생독립운동 광주학생독립운동을 광주학생독립운동의 광주학생독립운동이 광주학생독립운...,article1
2,2023-11-01,[소감문] “조국 위해 분연히 일어났던 선조들의 뜻 가슴에 새길래요”,무등일보는 지난 27일 광주서부교육지원청과 공동으로 제94주년 학생의 날을 맞아 광...,http://www.mdilbo.com/detail/csATmn/705817,광주학생독립운동의 광주학생독립운동에 광주학생독립운동은 광주학생독립운동가들의 광주학생...,article2
3,2023-11-01,"이승만 양자 이인수 박사 별세 ""父 뜻처럼 北동포 자유 누리길""",이승만 전 대통령 양자인 이인수 박사가 지난 9월 1일 오전 서울 강북구 수유동 4...,https://www.joongang.co.kr/article/25204088,이승만건국대통령기념사업회는 대통령은 이승만대통령기념관 대한민국에서 충청북도 이승만대...,article3
4,2023-11-01,JMS 정명석 ‘법관 기피 신청’ 최종 기각 재판 재개 전망,기독교복음선교회(일명 JMS) 총재인 정명석(왼쪽)과 JMS 2인자로 알려진 김지선...,https://www.khan.co.kr/national/national-gener...,대법원에서 jms 재판부를 대법원 대법원은 재판부는 기독교복음선교회 재판부의 대법원...,article4


In [ ]:
df.to_excel('/content/drive/MyDrive/Colab Notebooks/Hackathon/keword추출(id포함).xlsx', index = None)

# Kiwi 형태소 분석 + kobert 핵심어 추출


`-` 키워드 추출 좀 더 보기 좋게 시도

1.   형태소 분석기 활용
2.   사용자 사전 추가?





In [ ]:
!pip install kiwipiepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 27.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kiwipiepy-model: filename=kiwipiepy_model-0.16.0-py3-none-any.whl size=30813255 sha256=bc4058bfe3bc3d2c370873b2054cc5147871a6bf72422ddb6ad510c5991f2f6d
  Stored in directory: /root/.cache/pip/wheels/6b/03/2b/ac2c97cc65ebd9df3516f4b900adc2f0a744df8d1375b2e2ef
Successfully built kiwipiepy-model


In [ ]:
from kiwipiepy import Kiwi
kiwi = Kiwi()
kiwi.analyze(text)
# kiwi.load_user_dictionary('user_dictionary.txt')
# kiwi.prepare()

[([Token(form='“', tag='SSO', start=0, len=1),
   Token(form='아들', tag='NNG', start=1, len=2),
   Token(form='에게', tag='JKB', start=3, len=2),
   Token(form='자랑', tag='NNG', start=6, len=2),
   Token(form='스럽', tag='XSA-I', start=8, len=3),
   Token(form='은', tag='ETM', start=10, len=1),
   Token(form='아버지', tag='NNG', start=12, len=3),
   Token(form='가', tag='JKC', start=15, len=1),
   Token(form='되', tag='VV', start=17, len=1),
   Token(form='겠', tag='EP', start=18, len=1),
   Token(form='다', tag='EF', start=19, len=1),
   Token(form='”', tag='SSC', start=20, len=1),
   Token(form='백악관', tag='NNP', start=22, len=3),
   Token(form='집무실', tag='NNG', start=26, len=3),
   Token(form='에', tag='JKB', start=29, len=1),
   Token(form='놓이', tag='VV', start=31, len=2),
   Token(form='ᆫ', tag='ETM', start=32, len=1),
   Token(form='사진', tag='NNG', start=34, len=2),
   Token(form='에', tag='JKB', start=36, len=1),
   Token(form='담기', tag='VV', start=38, len=2),
   Token(form='ᆫ', tag='ETM', start

In [ ]:
def noun_extractor(df):
    results = []

    for i in range(len(df)):
        text = df["뉴스 본문"][i]
        result = kiwi.analyze(text)
        tokens = []
        for token, pos, _, _ in result[0][0]:
            if len(token) != 1 and (pos.startswith('N') or pos.startswith('SL')):
                tokens.append("".join(token))
        results.append(tokens)

    df["형태소분석"] = results
    df["형태소분석"] = [" ".join(i) for i in df["형태소분석"]]

    return df

In [ ]:
df = noun_extractor(df)
df.head(3)

,일자,제목,뉴스 본문,URL,형태소분석
0,2023-11-01,"군복 입은 아들의 사진, 이 대통령을 강직하게 만들었다[정미경의 이런영어 저런미국]",“아들에게 자랑스러운 아버지가 되겠다” 백악관 집무실에 놓인 사진에 담긴 사연 대통...,https://www.donga.com/news/Inter/article/all/2...,아들 자랑 아버지 백악관 집무실 사진 사연 대통령 오피스 인테리어 감각 바이든 미국...
1,2023-11-01,"""광주학생들의 저항정신 곳곳에 살아 숨 쉰다""",광주서부교육지원청과 무등일보가 공동 주최한 94주년 광주학생독립운동 발상지 현장취재...,http://www.mdilbo.com/detail/csATmn/705819,광주 서부교육지원청 무등일보 공동 주최 주년 광주 학생 독립운동 발상지 현장 취재 ...
2,2023-11-01,[소감문] “조국 위해 분연히 일어났던 선조들의 뜻 가슴에 새길래요”,무등일보는 지난 27일 광주서부교육지원청과 공동으로 제94주년 학생의 날을 맞아 광...,http://www.mdilbo.com/detail/csATmn/705817,무등 일보 광주 서부교육지원청 공동 주년 학생 광주 학생 독립운동 정신 계승 기자 ...


In [ ]:
def keyword_extract(df):
    model = BartForConditionalGeneration.from_pretrained('digit82/kobart-summarization')
    kw_model = KeyBERT(model)

    keyword_list = []

    for i in range(len(df)):
        text = df["형태소분석"][i]
        keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=20)
        temp = [keyword[0] for keyword in keywords]
        keyword_list.append(temp)

    df["keyword"] = keyword_list
    df["keyword"] = [",".join(keyword) for keyword in df["keyword"]]

    return df

In [ ]:
input_df.to_excel('/content/drive/MyDrive/Colab Notebooks/Hackathon/형태소분석포함.xlsx', index = None)